# Capstone project of the Coursera

In [9]:
import requests
import pandas as pd
import numpy as np
import random

from geopy.geocoders import Nominatim

from IPython.display import Image
from IPython.core.display import HTML

from pandas.io.json import json_normalize

import folium

I have used the fourquare RESTful APIs to collect the information about the location
1. App settings -> Redirect URL -> Web Addresses-> https://www.google.com

2. To get access token:

3. Replace it with the client_id https://foursquare.com/oauth2/authenticate?client_id=YOUR_CLIENT_ID&response_type=code&redirect_uri=https://www.google.com

4. It should look like https://www.google.com/?code=CODE take the code part

5. Replace all client_id, sceret and code to get the access token https://foursquare.com/oauth2/access_token?client_id=YOUR_CLIENT_ID&client_secret=YOUR_CLIENT_SECRET&grant_type=authorization_code&redirect_uri=https://www.google.com&code=CODE. 

In [10]:
CLIENT_ID = 'NX2GTOOXPXMNZGR2FQP4COOV3NPCYVWMVKCTCJB54CLEOL0N' # your Foursquare ID
CLIENT_SECRET = 'WIFEAP2OAFGGR1OGTQOHMQWHBP3ZD1ZQM2S42K0FPG3BO0MJ' # your Foursquare Secret
ACCESS_TOKEN = '' # your FourSquare Access Token
VERSION = '20210201'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NX2GTOOXPXMNZGR2FQP4COOV3NPCYVWMVKCTCJB54CLEOL0N
CLIENT_SECRET:WIFEAP2OAFGGR1OGTQOHMQWHBP3ZD1ZQM2S42K0FPG3BO0MJ


In [17]:
address = '102 North End Ave, New York, NY'

geolocator= Nominatim(user_agent="foursquare_agent")
location=geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7149555 -74.0153365


# 1. Search for a specific venue category

    https://api.foursquare.com/v2/venues/search?client_id=CLIENT_ID&client_secret=CLIENT_SECRET&ll=LATITUDE,LONGITUDE&v=VERSION&query=QUERY&radius=RADIUS&limit=LIMIT

In [18]:
search_query = 'Italian'
radius = 500
print(search_query + ' .... OK!')

Italian .... OK!


In [23]:
url="https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&query={}&limit={}".format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION,radius,search_query, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=NX2GTOOXPXMNZGR2FQP4COOV3NPCYVWMVKCTCJB54CLEOL0N&client_secret=WIFEAP2OAFGGR1OGTQOHMQWHBP3ZD1ZQM2S42K0FPG3BO0MJ&ll=40.7149555,-74.0153365&v=20210201&radius=500&query=Italian&limit=30'

In [24]:
results=requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '601954c8f3028a465bfc3e4b'},
 'response': {'venues': [{'id': '4fa862b3e4b0ebff2f749f06',
    'name': "Harry's Italian Pizza Bar",
    'location': {'address': '225 Murray St',
     'lat': 40.71521779064671,
     'lng': -74.01473940209351,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.71521779064671,
       'lng': -74.01473940209351},
      {'label': 'entrance', 'lat': 40.715361, 'lng': -74.014975}],
     'distance': 58,
     'postalCode': '10282',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['225 Murray St',
      'New York, NY 10282',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1ca941735',
      'name': 'Pizza Place',
      'pluralName': 'Pizza Places',
      'shortName': 'Pizza',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-16122728

In [56]:
venues=results['response']['venues']

df=pd.json_normalize(venues)
df

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet
0,4fa862b3e4b0ebff2f749f06,Harry's Italian Pizza Bar,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1612272840,False,225 Murray St,40.715218,-74.014739,"[{'label': 'display', 'lat': 40.71521779064671...",58,10282,US,New York,NY,United States,"[225 Murray St, New York, NY 10282, United Sta...",NaN
1,4f3232e219836c91c7bfde94,Conca Cucina Italian Restaurant,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F...",v-1612272840,False,63 W Broadway,40.714484,-74.009806,"[{'label': 'display', 'lat': 40.71448400000000...",469,10007,US,New York,NY,United States,"[63 W Broadway, New York, NY 10007, United Sta...",NaN
2,3fd66200f964a520f4e41ee3,Ecco,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1612272840,False,124 Chambers St,40.715337,-74.008848,"[{'label': 'display', 'lat': 40.71533713859952...",549,10007,US,New York,NY,United States,[124 Chambers St (btwn Church St & W Broadway)...,btwn Church St & W Broadway


In [55]:
filtered_columns=['name','categories']+[col for col in df.columns if col.startswith ('location.')]+['id']
df_filtered=df.loc[:,filtered_columns]

df_filtered.columns = [column.split('.')[-1] for column in df_filtered.columns]
 
def get_cat(row):
        cat = row['categories']
    if len(cat)==0:
        return None
    else:
        return cat[0]['name']

df_filtered['categories']=df.apply(get_cat,axis=1)

df_filtered
                                                                                                

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Harry's Italian Pizza Bar,Pizza Place,225 Murray St,40.715218,-74.014739,"[{'label': 'display', 'lat': 40.71521779064671...",58,10282,US,New York,NY,United States,"[225 Murray St, New York, NY 10282, United Sta...",NaN,4fa862b3e4b0ebff2f749f06
1,Conca Cucina Italian Restaurant,Food,63 W Broadway,40.714484,-74.009806,"[{'label': 'display', 'lat': 40.71448400000000...",469,10007,US,New York,NY,United States,"[63 W Broadway, New York, NY 10007, United Sta...",NaN,4f3232e219836c91c7bfde94
2,Ecco,Italian Restaurant,124 Chambers St,40.715337,-74.008848,"[{'label': 'display', 'lat': 40.71533713859952...",549,10007,US,New York,NY,United States,[124 Chambers St (btwn Church St & W Broadway)...,btwn Church St & W Broadway,3fd66200f964a520f4e41ee3


In [57]:
venues_map=folium.Map(location=[latitude,longitude],zoom_start=13)

folium.CircleMarker(
    [latitude,longitude],
    radius=10,
    color='red',
    popup='Corad Hotel',
    fill=True,
    fill_color='red',
    fill_opacity=0.6
).add_to(venues_map)

for lat,lon,label in zip(df_filtered.lat,df_filtered.lng,df_filtered.categories):
    folium.CircleMarker(
    [lat,lon],
    radius=10,
    color='red',
    popup=label,
    fill=True,
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)
    

venues_map

# 2. Explore a Given Venue

    https://api.foursquare.com/v2/venues/VENUE_ID?client_id=CLIENT_ID&client_secret=CLIENT_SECRET&v=VERSION



In [60]:
venue_id = '4fa862b3e4b0ebff2f749f06' # ID of Harry's Italian Pizza Bar
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4fa862b3e4b0ebff2f749f06?client_id=NX2GTOOXPXMNZGR2FQP4COOV3NPCYVWMVKCTCJB54CLEOL0N&client_secret=WIFEAP2OAFGGR1OGTQOHMQWHBP3ZD1ZQM2S42K0FPG3BO0MJ&v=20210201'

In [62]:
results=requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60195ed34b515236ca78caec'},
 'response': {'venue': {'id': '4fa862b3e4b0ebff2f749f06',
   'name': "Harry's Italian Pizza Bar",
   'contact': {'phone': '2126081007', 'formattedPhone': '(212) 608-1007'},
   'location': {'address': '225 Murray St',
    'lat': 40.71521779064671,
    'lng': -74.01473940209351,
    'labeledLatLngs': [{'label': 'display',
      'lat': 40.71521779064671,
      'lng': -74.01473940209351},
     {'label': 'entrance', 'lat': 40.715361, 'lng': -74.014975}],
    'postalCode': '10282',
    'cc': 'US',
    'city': 'New York',
    'state': 'NY',
    'country': 'United States',
    'formattedAddress': ['225 Murray St',
     'New York, NY 10282',
     'United States']},
   'canonicalUrl': 'https://foursquare.com/v/harrys-italian-pizza-bar/4fa862b3e4b0ebff2f749f06',
   'categories': [{'id': '4bf58dd8d48988d1ca941735',
     'name': 'Pizza Place',
     'pluralName': 'Pizza Places',
     'shortName': 'Pizza',
     'icon': {'prefix': 'https

In [64]:
try:
    print(results['response']['venue']['rating'])
except:
    print("no results found")

6.9


In [65]:
venue_id = '4f3232e219836c91c7bfde94'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

results=requests.get(url).json()

try:
    print(results['response']['venue']['rating'])
except:
    print("no results found")

no results found


In [66]:
venue_id = '3fd66200f964a520f4e41ee3' # ID of Ecco
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

results=requests.get(url).json()

try:
    print(results['response']['venue']['rating'])
except:
    print("no results found")

7.3


In [68]:
results['response']['venue']['tips']['count']

19

In [69]:
limit = 15 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6019935b2ea7ee340a485844'},
 'response': {'tips': {'count': 19,
   'items': [{'id': '5ab1cb46c9a517174651d3fe',
     'createdAt': 1521601350,
     'text': 'A+ Italian food! Trust me on this: my mom’s side of the family is 100% Italian. I was born and bred to know good pasta when I see it, and Ecco is one of my all-time NYC favorites',
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/5ab1cb46c9a517174651d3fe',
     'lang': 'en',
     'likes': {'count': 0, 'groups': []},
     'logView': True,
     'agreeCount': 5,
     'disagreeCount': 0,
     'todo': {'count': 0},
     'user': {'firstName': 'Nick',
      'lastName': 'E',
      'address': '',
      'city': '',
      'state': ''},
     'authorInteractionType': 'liked'}]}}}

In [70]:
tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user', 'authorInteractionType'])